In [1]:
import json
import pandas as pd
import numpy as np

pasta = 'elfa/'

In [2]:
dados_input = json.load(open(f"{pasta}meta_dados.json", "r", encoding="cp1252"))
dados = dados_input["dados"]
metadados = dados_input["metadados"]

dados.keys(), metadados.keys()

(dict_keys(['criacaoDeNiveis', 'operacaoEntreNiveis', 'restricaoEntreUfs', 'tipoDeTransporte', 'criacaoDeClusterDeNos', 'criacaoDeNos', 'atribuicaoCriacaoDeNos', 'criacaoDeCluster', 'skus', 'clustersSkuAtribuicao', 'baseline', 'demandaPorSku', 'restricoesDeSingleSource', 'restricoesParaClustersDeNos', 'no', 'noClusterDeSku', 'noSku', 'dimensionamentoArmazenagem', 'freteBase', 'fretePorSku', 'fretePorCluster', 'ltDaRota', 'kmDaRota', 'configNosNS', 'nivelDeServico', 'nivelDeServicoPorCluster', 'nivelDeServicoPorSku', 'fatorRendimentoSku', 'fatorConsumoSku', 'fatorDeGiroCluNoCluSku', 'custoCapEstCluNo', 'porOperacao', 'porRota', 'valorDeVenda', 'icmsFcp', 'icmsPorUf', 'icmsPorNo', 'debitoIcmsNaEntradaUf', 'debitoIcmsNaEntradaPorNo', 'icmsAproveitamentoCredito', 'icmsAproveitamentoDebito', 'anulacaoDeCreditoIcmsUf', 'anulacaoDeCreditoIcmsNo', 'iss', 'impostosFederais', 'impostosFederaisPorCluster', 'impostosFederaisPorSku', 'icmsSt', 'icmsStClusterDeNo', 'icmsStNo', 'antecipacaoIcms', 'cr

In [3]:
nos = pd.DataFrame(dados["criacaoDeNos"])
# criar coluna id, que corresponde ao nivel do no e um numero sequencial iniciando em 1 para cada nivel
nos["idn"] = nos.groupby("nivel").cumcount() + 1
nos["idn"] = nos["nivel"] + "_" + nos["idn"].astype(str)
replace_nos = dict(zip(nos["no"], nos["idn"]))
nos.to_csv(f"{pasta}nos.csv", index=False)
nos.drop(columns=["no"]).to_csv(f"{pasta}nos_anonim.csv", index=False)
nos

,no,nivel,uf,idn
0,F,Fornecedor,SP,Fornecedor_1
1,CD_SP2,CD,SP,CD_1
2,CD_SP1,CD,SP,CD_2
3,CD_DF,CD,DF,CD_3
4,CD_MG,CD,MG,CD_4
...,...,...,...,...
174,TO_Capital_Público_Não Contribuinte_Geral_Não ...,Cliente,TO,Cliente_159
175,TO_Interior_Privado_Contribuinte_Geral_Não Est...,Cliente,TO,Cliente_160
176,TO_Interior_Privado_Não Contribuinte_Geral_Não...,Cliente,TO,Cliente_161
177,TO_Interior_Público_Não Contribuinte_Geral_Não...,Cliente,TO,Cliente_162


In [4]:
skus = pd.DataFrame(dados["skus"])
skus["ids"] = "sku_" + skus.index.astype(str)
skus.to_csv(f"{pasta}skus.csv", index=False)
replace_skus = dict(zip(skus["listaDeSkus"], skus["ids"]))
skus.drop(columns=["listaDeSkus"]).to_csv(f"{pasta}skus_anonim.csv", index=False)
skus

,listaDeSkus,ids
0,ESPECIALIDADES_162 Importado_NNC_S,sku_0
1,ESPECIALIDADES_162 Importado_SS_R2,sku_1
2,ESPECIALIDADES_162 Importado_SS_S,sku_2
3,ESPECIALIDADES_162 Nacional_SS_R2,sku_3
4,ESPECIALIDADES_162 Nacional_SS_S,sku_4
...,...,...
89,MATERIAIS_Tributado Importado_NNC_S_RE,sku_89
90,NUTRICAO_Isento_SS_S_RE,sku_90
91,NUTRICAO_Tributado Importado_NNC_S_RE,sku_91
92,NUTRICAO_Tributado Nacional_NNC_S_RE,sku_92


In [9]:
frete = pd.DataFrame(dados["freteBase"])
if not frete.empty:
    frete["origem"] = frete["origem"].replace(replace_nos)
    frete["destino"] = frete["destino"].replace(replace_nos)
    frete.to_csv(f"{pasta}frete_base.csv", index=False)
    print(frete.head())

frete = pd.DataFrame(dados["fretePorCluster"])
if not frete.empty:
    frete["origem"] = frete["origem"].replace(replace_nos)
    frete["destino"] = frete["destino"].replace(replace_nos)
    frete.to_csv(f"{pasta}frete_cluster.csv", index=False)
    print(frete.head())


frete = pd.DataFrame(dados["fretePorSku"])
if not frete.empty:
    frete["origem"] = frete["origem"].replace(replace_nos)
    frete["destino"] = frete["destino"].replace(replace_nos)
    frete["sku"] = frete["sku"].replace(replace_skus)
    frete.to_csv(f"{pasta}frete_sku.csv", index=False)
    print(frete.head())

         origem destino         cluster tipoDeFrete      perfil  custo  \
0  Fornecedor_1   CD_11  ESPECIALIDADES     Inbound  Rodoviario    0.0   
1  Fornecedor_1    CD_7  ESPECIALIDADES     Inbound  Rodoviario    0.0   
2  Fornecedor_1    CD_3  ESPECIALIDADES     Inbound  Rodoviario    0.0   
3  Fornecedor_1    CD_5  ESPECIALIDADES     Inbound  Rodoviario    0.0   
4  Fornecedor_1    CD_4  ESPECIALIDADES     Inbound  Rodoviario    0.0   

   volumeMinimo  forcarUso  desconsiderarCustoFrete  debitoIcmsOrigem  \
0           0.0          3                    False             False   
1           0.0          3                    False             False   
2           0.0          3                    False             False   
3           0.0          3                    False             False   
4           0.0          3                    False             False   

   creditoIcmsDestino  
0               False  
1               False  
2               False  
3               Fals

In [10]:
km_rota = pd.DataFrame(dados["kmDaRota"])
km_rota["origem"] = km_rota["origem"].replace(replace_nos)
km_rota["destino"] = km_rota["destino"].replace(replace_nos)
km_rota.to_csv(f"{pasta}km_rota.csv", index=False)
km_rota

,origem,destino,distanciaRotaKM
0,Fornecedor_1,CD_11,10
1,Fornecedor_1,CD_7,10
2,Fornecedor_1,CD_3,10
3,Fornecedor_1,CD_5,10
4,Fornecedor_1,CD_4,10
...,...,...,...
2455,CD_9,Cliente_163,"1247,224462678961"
2456,CD_10,Cliente_163,"1013,846684219732"
2457,CD_2,Cliente_163,"255,8834881919303"
2458,CD_1,Cliente_163,"224,03470650081258"


In [11]:
baseline = pd.DataFrame(dados["baseline"])
baseline["origem"] = baseline["origem"].replace(replace_nos)
baseline["destino"] = baseline["destino"].replace(replace_nos)
baseline["sku"] = baseline["sku"].replace(replace_skus)
baseline.to_csv(f"{pasta}baseline.csv", index=False)
baseline

,origem,destino,sku,perfil,volume
0,CD_11,Cliente_2,sku_82,RODOVIARIO,246.57
1,CD_11,Cliente_5,sku_40,RODOVIARIO,805.64
2,CD_11,Cliente_5,sku_43,RODOVIARIO,1077.92
3,CD_11,Cliente_5,sku_47,RODOVIARIO,780.03
4,CD_11,Cliente_5,sku_52,RODOVIARIO,34591.75
...,...,...,...,...,...
10882,Fornecedor_1,CD_1,sku_84,RODOVIARIO,37696342.98
10883,Fornecedor_1,CD_1,sku_85,RODOVIARIO,17937.41
10884,Fornecedor_1,CD_1,sku_86,RODOVIARIO,487841.21
10885,Fornecedor_1,CD_1,sku_88,RODOVIARIO,10114.48


In [12]:
valor_mercadoria = pd.DataFrame(dados["porRota"])
valor_mercadoria["origem"] = valor_mercadoria["origem"].replace(replace_nos)
valor_mercadoria["destino"] = valor_mercadoria["destino"].replace(replace_nos)
valor_mercadoria["sku"] = valor_mercadoria["sku"].replace(replace_skus)
valor_mercadoria.to_csv(f"{pasta}valor_mercadoria.csv", index=False)
valor_mercadoria

,origem,destino,sku,valorMercadoria
0,CD_11,Cliente_1,sku_49,9.37
1,CD_11,Cliente_2,sku_49,9.37
2,CD_11,Cliente_3,sku_49,9.37
3,CD_11,Cliente_4,sku_49,9.37
4,CD_11,Cliente_5,sku_49,9.37
...,...,...,...,...
231235,Fornecedor_1,CD_15,sku_92,35.06
231236,Fornecedor_1,CD_15,sku_86,1.07
231237,Fornecedor_1,CD_15,sku_87,42.03
231238,Fornecedor_1,CD_15,sku_91,10.84


In [13]:
demanda = pd.DataFrame(dados["demandaPorSku"])
demanda["sku"] = demanda["sku"].replace(replace_skus)
demanda["no"] = demanda["no"].replace(replace_nos)
demanda.to_csv(f"{pasta}demanda.csv", index=False)
demanda

,no,sku,perfil,demanda
0,Cliente_1,sku_82,Rodoviário,0.20
1,Cliente_2,sku_0,Rodoviário,65.24
2,Cliente_2,sku_2,Rodoviário,271.32
3,Cliente_2,sku_6,Rodoviário,0.30
4,Cliente_2,sku_8,Rodoviário,0.10
...,...,...,...,...
4089,Cliente_161,sku_84,Rodoviário,53.41
4090,Cliente_161,sku_88,Rodoviário,1591.67
4091,Cliente_162,sku_18,Rodoviário,53.87
4092,Cliente_162,sku_35,Rodoviário,16.08


In [14]:
custo_nos = pd.DataFrame(dados["no"])
custo_nos["no"] = custo_nos["no"].replace(replace_nos)
custo_nos.to_csv(f"{pasta}custo_nos.csv", index=False)
custo_nos

,no,capFornecimento,capExpedicao,custoFixo,custoVariavelExpedicao,forcarUso,volumeMinimo
0,Fornecedor_1,1.000000e+09,1.000000e+09,10.00,0.0,3,0.0
1,CD_11,1.000000e+09,3.782349e+08,1923708.73,0.0,3,0.0
2,CD_7,1.000000e+09,3.782349e+08,1695833.18,0.0,3,0.0
3,CD_3,1.000000e+09,3.782349e+08,3677572.50,0.0,3,0.0
4,CD_5,1.000000e+09,3.782349e+08,2785926.28,0.0,3,0.0
...,...,...,...,...,...,...,...
224,SE_PRIVADO,1.000000e+09,3.782349e+08,10.00,0.0,3,0.0
225,SP_PUBLICO,1.000000e+09,3.782349e+08,10.00,0.0,3,0.0
226,SP_PRIVADO,1.000000e+09,3.782349e+08,10.00,0.0,3,0.0
227,TO_PUBLICO,1.000000e+09,3.782349e+08,10.00,0.0,3,0.0
